In [28]:
pip install -U deepctr==0.8.1

     |████████████████████████████████| 110 kB 34.4 MB/s            
  Attempting uninstall: deepctr
    Found existing installation: deepctr 0.9.0
    Uninstalling deepctr-0.9.0:
      Successfully uninstalled deepctr-0.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepmatch 0.2.0 requires deepctr==0.8.2, but you have deepctr 0.8.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import deepctr
deepctr.__version__

'0.8.1'

In [3]:
# pip install deepmatch


In [2]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.utils.vis_utils import plot_model

In [3]:
import pandas as pd
masterdf = pd.read_csv('s3a://hluan/hm/sampled_10_users_transactions.csv', dtype={"article_id": "str"})

In [4]:
from datetime import datetime
masterdf['t_dat'] = pd.to_datetime(masterdf['t_dat'])

In [5]:
masterdf['t_dat'] = datetime(2020, 9, 22) - masterdf['t_dat']
masterdf['t_dat'] = (masterdf['t_dat'].dt.days / 7).astype('int')

In [6]:
masterdf['t_dat'] = 104 - masterdf['t_dat']

In [81]:
master_df_part = masterdf[['t_dat', 'customer_id', 'article_id', 'price']]

In [82]:
master_df_part = master_df_part[master_df_part['t_dat'] > 50]

In [83]:
master_df_part['rating'] = 1

In [84]:
interactions_dict = master_df_part.groupby(['customer_id', 
                                      'article_id',
                                      't_dat'])[['rating', 'price']].sum().reset_index()

In [85]:
interactions_dict.shape

(1369514, 5)

In [86]:
input_data_path = ""
customers = pd.read_csv(f'{input_data_path}customers.csv', dtype={"article_id": "str"})
articles = pd.read_csv(f'{input_data_path}articles.csv', dtype={"article_id": "str"})

In [87]:
articles_processed = articles[['article_id', 
       'prod_name', 
       'product_type_name', 'product_group_name', 
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name',
       'perceived_colour_master_name',
       'department_name', 'index_name',
       'index_group_name', 'section_name',
       'garment_group_name']]

In [88]:
bins = [0, 35, 45, 55, 120]
labels = [1,2,3,4]
customers['age_group'] = pd.cut(customers['age'], bins=bins, labels=labels)

In [89]:
interactions_dict = interactions_dict[['t_dat', 'customer_id', 'article_id', 'rating', 'price']]\
    .merge(articles_processed, on='article_id', how='left')\
    .merge(customers[['customer_id', 'age_group']], on='customer_id', how='left')\

In [90]:
interactions_dict['age_group'] = interactions_dict['age_group'].fillna(1)


In [91]:
interactions_dict.columns

Index(['t_dat', 'customer_id', 'article_id', 'rating', 'price', 'prod_name',
       'product_type_name', 'product_group_name', 'graphical_appearance_name',
       'colour_group_name', 'perceived_colour_value_name',
       'perceived_colour_master_name', 'department_name', 'index_name',
       'index_group_name', 'section_name', 'garment_group_name', 'age_group'],
      dtype='object')

In [92]:
interactions_dict.isnull().sum()

t_dat                           0
customer_id                     0
article_id                      0
rating                          0
price                           0
prod_name                       0
product_type_name               0
product_group_name              0
graphical_appearance_name       0
colour_group_name               0
perceived_colour_value_name     0
perceived_colour_master_name    0
department_name                 0
index_name                      0
index_group_name                0
section_name                    0
garment_group_name              0
age_group                       0
dtype: int64

In [93]:
import random
import numpy as np
from tqdm import tqdm
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

def gen_data_set(data, negsample=0):

    data.sort_values("t_dat", inplace=True)
    item_ids = data['article_id'].unique()

    train_set = []
    test_set = []
    for reviewerID, hist in tqdm(data.groupby('customer_id')):
        pos_list = hist['article_id'].tolist()
        rating_list = hist['rating'].tolist()

        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            if i != len(pos_list) - 1:
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1, len(hist[::-1]),rating_list[i]))
                for negi in range(negsample):
                    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1])))
            else:
                test_set.append((reviewerID, hist[::-1], pos_list[i],1,len(hist[::-1]),rating_list[i]))

    random.shuffle(train_set)
    random.shuffle(test_set)

    print(len(train_set[0]),len(test_set[0]))

    return train_set,test_set

In [94]:
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.models import Model

from deepmatch.models import *

In [95]:
data = interactions_dict.sample(frac=0.1)
sparse_features = ['customer_id', 'article_id', 'sales_channel_id', 'FN',
       'Active', 'club_member_status', 'fashion_news_frequency', 'age',
       'postal_code', 'product_code', 'product_type_no', 'product_group_code',
       'graphical_appearance_no', 'colour_group_code',
       'perceived_colour_value_id', 'perceived_colour_master_id',
       'department_no', 'index_code_id', 'index_group_no', 'section_no',
       'garment_group_no',]
SEQ_LEN = 30
negsample = 1

# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`

features = ['customer_id', 'article_id', 'rating', 'prod_name',
       'product_type_name', 'product_group_name', 'graphical_appearance_name',
       'colour_group_name', 'perceived_colour_value_name',
       'perceived_colour_master_name', 'department_name', 'index_name',
       'index_group_name', 'section_name', 'garment_group_name', 'age_group']
feature_max_idx = {}
for feature in features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1



item_profile = data[['article_id', 'prod_name',
       'product_type_name', 'product_group_name', 'graphical_appearance_name',
       'colour_group_name', 'perceived_colour_value_name',
       'perceived_colour_master_name', 'department_name', 'index_name',
       'index_group_name', 'section_name', 'garment_group_name']].drop_duplicates('article_id')


user_item_list = data.groupby("customer_id")['article_id'].apply(list)

train_set, test_set = gen_data_set(data, negsample)

100%|██████████| 54060/54060 [05:07<00:00, 175.98it/s]


5 6


In [64]:
data

,t_dat,customer_id,article_id,rating,price,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name,age_group
76381,91,4470,8178,1,0.013542,7135,86,5,24,2,2,2,23,5,2,12,6,1
141756,91,8370,1140,1,0.011847,3633,71,4,7,3,6,3,147,7,3,43,12,1
12712,91,715,50,1,0.022017,305,25,3,24,2,2,2,23,5,2,12,6,1
310496,91,18349,4215,1,0.008458,1502,86,5,24,46,4,16,69,7,3,42,6,2
232671,91,13752,9785,1,0.021593,5233,9,5,24,47,5,17,28,7,3,41,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393390,104,23293,617,1,0.016932,4910,86,5,24,6,2,3,69,7,3,42,6,1
323915,104,19148,12032,4,0.271119,7162,91,4,7,2,2,2,38,5,2,26,18,1
234869,104,13869,7661,1,0.050831,991,73,4,24,6,2,3,150,7,3,51,13,1
109518,104,6450,11566,1,0.033881,7099,9,5,24,2,2,2,28,7,3,43,2,1


In [22]:
train_set[0]

(14000, [7294], 9718, 0, 1)

In [23]:
np.save('neg_sampled_train_5_sampled_10', train_set)
np.save('neg_sampled_test_5_sampled_10', test_set)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [37]:
train_set = np.load('neg_sampled_train_5_sampled.npy', allow_pickle=True)

In [38]:
test_set = np.load('neg_sampled_test_5_sampled.npy', allow_pickle=True)

In [ ]:
len(test_set)

In [24]:
train_set[3]

(30094, [26302, 21516, 874, 2552], 27236, 1, 4, 2)

In [96]:
def gen_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {"customer_id": train_uid, "article_id": train_iid, "hist_article_id": train_seq_pad,
                         "hist_len": train_hist_len}

    for key in ['prod_name',
       'product_type_name', 'product_group_name', 'graphical_appearance_name',
       'colour_group_name', 'perceived_colour_value_name',
       'perceived_colour_master_name', 'department_name', 'index_name',
       'index_group_name', 'section_name', 'garment_group_name', 'age_group']:
        train_model_input[key] = user_profile.loc[train_model_input['customer_id']][key].values

    return train_model_input, train_label

In [97]:
user_profile = data[['customer_id', 'age_group', 'prod_name',
       'product_type_name', 'product_group_name', 'graphical_appearance_name',
       'colour_group_name', 'perceived_colour_value_name',
       'perceived_colour_master_name', 'department_name', 'index_name',
       'index_group_name', 'section_name', 'garment_group_name']].drop_duplicates('customer_id')

In [98]:
user_profile.set_index("customer_id", inplace=True)


In [99]:
user_profile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54060 entries, 40007 to 16136
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   age_group                     54060 non-null  int64
 1   prod_name                     54060 non-null  int64
 2   product_type_name             54060 non-null  int64
 3   product_group_name            54060 non-null  int64
 4   graphical_appearance_name     54060 non-null  int64
 5   colour_group_name             54060 non-null  int64
 6   perceived_colour_value_name   54060 non-null  int64
 7   perceived_colour_master_name  54060 non-null  int64
 8   department_name               54060 non-null  int64
 9   index_name                    54060 non-null  int64
 10  index_group_name              54060 non-null  int64
 11  section_name                  54060 non-null  int64
 12  garment_group_name            54060 non-null  int64
dtypes: int64(13)
memory usage: 

In [100]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
# test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)



In [101]:
user_profile

,age_group,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name
customer_id,,,,,,,,,,,,,
40007,1,3080,95,7,26,9,6,4,78,7,3,43,6
10445,1,774,8,13,1,47,4,16,183,8,3,51,16
45556,1,13999,95,7,26,2,2,2,26,5,2,12,6
50126,1,5129,105,14,26,2,2,2,163,8,3,46,19
390,1,4077,28,5,27,30,4,16,78,7,3,43,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7364,3,13051,100,6,8,8,2,6,46,9,4,9,18
42201,1,6668,105,14,26,2,2,2,163,8,3,46,19
45856,1,3068,62,1,1,25,3,12,140,6,3,50,1


In [102]:
import pandas as pd
from deepctr.feature_column import SparseFeat,get_feature_names
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.models import Model
from deepmatch.models import *

embedding_dim = 32
SEQ_LEN = 30
user_feature_columns = [SparseFeat('customer_id', feature_max_idx["customer_id"], embedding_dim),
                        SparseFeat("age_group", feature_max_idx["age_group"], embedding_dim),
                        VarLenSparseFeat(SparseFeat('hist_article_id', feature_max_idx["article_id"], embedding_dim,
                                                    embedding_name="article_id"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

item_feature_columns = [SparseFeat(col, feature_max_idx[col], embedding_dim) for col in [
    'article_id', 'prod_name',
       'product_type_name', 'product_group_name', 'graphical_appearance_name',
       'colour_group_name', 'perceived_colour_value_name',
       'perceived_colour_master_name', 'department_name', 'index_name',
       'index_group_name', 'section_name', 'garment_group_name']]


In [103]:
from deepctr.feature_column import build_input_features
from deepctr.inputs import create_embedding_matrix
from deepctr.layers.utils import combined_dnn_input 
from deepctr.layers.core import PredictionLayer, DNN
from tensorflow.python.keras.models import Model
from deepmatch.inputs import input_from_feature_columns
from deepmatch.layers.core import Similarity


def DSSM(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(64, 32),
         item_dnn_hidden_units=(64, 32),
         dnn_activation='tanh', dnn_use_bn=False,
         l2_reg_dnn=0, l2_reg_embedding=1e-6, dnn_dropout=0, seed=1024, metric='cos'):
    """Instantiates the Deep Structured Semantic Model architecture.
    :param user_feature_columns: An iterable containing user's features used by  the model.
    :param item_feature_columns: An iterable containing item's features used by  the model.
    :param user_dnn_hidden_units: list,list of positive integer or empty list, the layer number and units in each layer of user tower
    :param item_dnn_hidden_units: list,list of positive integer or empty list, the layer number and units in each layer of item tower
    :param dnn_activation: Activation function to use in deep net
    :param dnn_use_bn: bool. Whether use BatchNormalization before activation or not in deep net
    :param l2_reg_dnn: float. L2 regularizer strength applied to DNN
    :param l2_reg_embedding: float. L2 regularizer strength applied to embedding vector
    :param dnn_dropout: float in [0,1), the probability we will drop out a given DNN coordinate.
    :param seed: integer ,to use as random seed.
    :param metric: str, ``"cos"`` for  cosine  or  ``"ip"`` for inner product
    :return: A Keras model instance.
    """

    embedding_matrix_dict = create_embedding_matrix(user_feature_columns + item_feature_columns, l2_reg_embedding,
                                                    seed=seed,
                                                    seq_mask_zero=True)

    user_features = build_input_features(user_feature_columns)
    user_inputs_list = list(user_features.values())
    user_sparse_embedding_list, user_dense_value_list = input_from_feature_columns(user_features,
                                                                                   user_feature_columns,
                                                                                   l2_reg_embedding, seed=seed,
                                                                                   embedding_matrix_dict=embedding_matrix_dict)
    user_dnn_input = combined_dnn_input(user_sparse_embedding_list, user_dense_value_list)

    item_features = build_input_features(item_feature_columns)
    item_inputs_list = list(item_features.values())
    item_sparse_embedding_list, item_dense_value_list = input_from_feature_columns(item_features,
                                                                                   item_feature_columns,
                                                                                   l2_reg_embedding, seed=seed,
                                                                                   embedding_matrix_dict=embedding_matrix_dict)
    item_dnn_input = combined_dnn_input(item_sparse_embedding_list, item_dense_value_list)

    user_dnn_out = DNN(user_dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                       dnn_use_bn, seed=seed)(user_dnn_input)

    item_dnn_out = DNN(item_dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                       dnn_use_bn, seed=seed)(item_dnn_input)

    score = Similarity(type=metric, gamma = 10)([user_dnn_out, item_dnn_out])

    output = PredictionLayer("binary", False)(score)

    model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

    model.__setattr__("user_input", user_inputs_list)
    model.__setattr__("item_input", item_inputs_list)
    model.__setattr__("user_embedding", user_dnn_out)
    model.__setattr__("item_embedding", item_dnn_out)

    return model

In [110]:
model = DSSM(user_feature_columns, item_feature_columns)
model.compile(optimizer='adagrad', loss="binary_crossentropy", metrics=['accuracy'])

history = model.fit(train_model_input, train_label,
                    batch_size=256, epochs=3, verbose=1, validation_split=0.2)

Epoch 1/3
335/335 [==============================] - 4s 9ms/step - loss: 0.7402 - accuracy: 0.4699 - val_loss: 0.7011 - val_accuracy: 0.4893
Epoch 2/3
335/335 [==============================] - 3s 8ms/step - loss: 0.6204 - accuracy: 0.6710 - val_loss: 0.6974 - val_accuracy: 0.5724
Epoch 3/3
335/335 [==============================] - 2s 7ms/step - loss: 0.5447 - accuracy: 0.7270 - val_loss: 0.7236 - val_accuracy: 0.5935


In [111]:
def get_test_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {"customer_id": train_uid, "hist_article_id": train_seq_pad,
                         "hist_len": train_hist_len}

    for key in ['age_group']:
        train_model_input[key] = user_profile.loc[train_model_input['customer_id']][key].values

    return train_model_input, train_label

In [112]:
item_profile.columns

Index(['article_id', 'prod_name', 'product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name'],
      dtype='object')

In [113]:
test_user_model_input, _ = get_test_model_input(test_set, user_profile, SEQ_LEN)
all_item_model_input = {col: item_profile[col].values for col in ['article_id', 'prod_name', 'product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name']}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

    

(29374, 32)
(28507, 32)


In [114]:
pip install faiss-cpu --no-cache

Note: you may need to restart the kernel to use updated packages.


In [116]:
# 5. [Optional] ANN search by faiss  and evaluate the result

test_true_label = {line[0]:[line[2]] for line in test_set}

import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(user_embs, 12)
s = []
hit = 0
for i, uid in tqdm(enumerate(test_user_model_input['customer_id'])):
    try:
        pred = [item_profile['article_id'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=12)
        s.append(recall_score)
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("recall", np.mean(s))
print("hr", hit / len(test_user_model_input['customer_id']))

29374it [00:01, 17865.50it/s]

recall 0.005378906515966501
hr 0.005378906515966501
